In [5]:
import numpy as np
import pandas as pd
import pandas.io.sql as psql
from datetime import datetime, timedelta, date
import argparse
import psycopg2
import psycopg2.extras as extras
import pickle

from math import *
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from db.postgres import *

In [30]:
PG_USER = "postgres"
PG_PASSWORD = "1"
PG_DATABASE = "stock"
PG_HOST = "172.18.0.3"
PG_PORT = 5432
PG_DRIVER = 'org.postgresql.Driver'

start_date = '2015-01-01'
end_date = '2023-02-01'

In [32]:
class DataHandler:

    '''
    '''

    def __init__(self, ticket, lookback = 60):
        self.ticket = ticket
        self.lookback = lookback


    def get_scaler(self):
        df = psql.read_sql(f"select date, close from stock_raw where stock = '{self.ticket}'", connect())
        dataset = df[['close']].values

        scaler = MinMaxScaler(feature_range=(0,1))
        dataset = scaler.fit_transform(dataset)

        self.dataset = dataset

        return scaler


    def model_input(self):

        self.get_scaler()

        dataX, dataY = [], []
        for i in range(len(self.dataset) - self.lookback):
            row = [a for a in self.dataset[i:i+self.lookback]]
            dataX.append(row)
            dataY.append(self.dataset[i + self.lookback][0])

        X, y = np.array(dataX), np.array(dataY)

        # Train-test split
        split_point = int(len(self.dataset)*0.9)
        X_train, y_train = X[:split_point], y[:split_point]
        X_val, y_val = X[split_point:len(X)], y[split_point:len(y)]

        return X_train, y_train, X_val, y_val

In [31]:
def load_model(ticket):
    return pickle.load(open(f'model/LTSM_{ticket}.sav', 'rb'))

def update_rmse(ticket, run_date):

    '''
    '''
    model = DataHandler(ticket=ticket, lookback=60)

    X_train, y_train, X_val, y_val = model.preprocess()

    ltsm = load_model(ticket)
    y_predict = ltsm.predict(X_val)

    scaler = model.get_scaler()
    predict_price = scaler.inverse_transform(y_predict.reshape(-1, 1))
    true_price = scaler.inverse_transform(y_val.reshape(-1, 1))

    rmse = sqrt(mean_squared_error(true_price, predict_price))

    postgres_operator(
        query=f"""
            delete from rmse where date = '{run_date}';
            insert into rmse values ({run_date}, {ticket}, {rmse});
        """,
        conn=connect()
    )

In [6]:
run_date = date.today()

data = psql.read_sql(f"""
            select close 
            from stock_raw 
            where stock = 'AAPL'
                and date between '{run_date - timedelta(days=90)}' and '{run_date}'
        """, connect())


Connect Postgres Successfully!!


In [17]:
t = data['close'].values
t

array([151.28999329, 148.00999451, 150.17999268, 151.07000732,
       148.11000061, 144.22000122, 141.16999817, 148.02999878,
       148.30999756, 147.80999756, 146.63000488, 142.91000366,
       140.94000244, 142.6499939 , 142.16000366, 144.49000549,
       145.47000122, 143.21000671, 136.5       , 134.50999451,
       132.36999512, 132.30000305, 135.44999695, 132.22999573,
       131.86000061, 130.02999878, 126.04000092, 129.61000061,
       129.92999268, 125.06999969, 126.36000061, 125.01999664,
       129.61999512, 130.1499939 , 130.72999573, 133.49000549,
       133.41000366, 134.75999451, 135.94000244, 135.21000671,
       135.27000427, 137.86999512, 141.11000061, 142.52999878,
       141.86000061, 143.96000671, 145.92999268, 143.        ,
       144.28999329, 145.42999268, 150.82000732, 154.5       ,
       151.72999573, 154.6499939 , 151.91999817, 150.86999512,
       151.00999451, 153.8500061 , 153.19999695, 155.33000183])

In [18]:
def transform_array(data, lookback=60):
        
    '''Transforming array into Model input'''
    temp = []
    for i in range(len(data) - lookback + 1):
        row = [[a] for a in data[i:i+lookback]]
        temp.append(row)
        
    return np.array(temp)

transform_array(t, 60)

array([[[151.28999329],
        [148.00999451],
        [150.17999268],
        [151.07000732],
        [148.11000061],
        [144.22000122],
        [141.16999817],
        [148.02999878],
        [148.30999756],
        [147.80999756],
        [146.63000488],
        [142.91000366],
        [140.94000244],
        [142.6499939 ],
        [142.16000366],
        [144.49000549],
        [145.47000122],
        [143.21000671],
        [136.5       ],
        [134.50999451],
        [132.36999512],
        [132.30000305],
        [135.44999695],
        [132.22999573],
        [131.86000061],
        [130.02999878],
        [126.04000092],
        [129.61000061],
        [129.92999268],
        [125.06999969],
        [126.36000061],
        [125.01999664],
        [129.61999512],
        [130.1499939 ],
        [130.72999573],
        [133.49000549],
        [133.41000366],
        [134.75999451],
        [135.94000244],
        [135.21000671],
        [135.27000427],
        [137.869